In [1]:
import os
import re

import cv2
from tqdm import tqdm

# Frame Extracter

In [ ]:
input_dir = r'E:\Dataset\Video'
output_dir = r'E:\Dataset\Frame'

In [ ]:
video_files = [f for f in os.listdir(input_dir)]

for video_name in video_files:
    video_path = os.path.join(input_dir, video_name)
    vid = cv2.VideoCapture(video_path)
    
    if not vid.isOpened():
        print(f"failed to open {video_name}")
        continue
    
    current_frame = 0
    sign_name = os.path.splitext(video_name)[0]
    data_folder = os.path.join(output_dir, sign_name)

    if not os.path.exists(data_folder):
        os.makedirs(data_folder)

    while (True):

        success, frame = vid.read()
        if not success:
            break
        
        # rotated_frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        
        cv2.imshow("output", frame)
        cv2.imwrite(os.path.join(data_folder, f"frame_{current_frame}.png"),
                                frame)
        current_frame += 1

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    vid.release()
    print(f"Completed:  {video_name} - {current_frame}")
    cv2.destroyAllWindows()

# Crop to 1:1

In [ ]:
base_dir = r'Dataset'

y1 = 420
y2 = 1500

In [ ]:
for folder in os.listdir((base_dir)):
        input_dir = os.path.join(base_dir, folder)
        if not os.path.isdir(input_dir):
                continue
        
        output_dir = os.path.join(base_dir, folder + "_cropped")
        
        if not os.path.exists(output_dir):
                os.makedirs(output_dir)
        

        img_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

        for filename in tqdm(img_files, desc=f"Processing {folder}", unit="file"):
                
                img_path = os.path.join(input_dir, filename)
                img = cv2.imread(img_path)
                
                if img is None:
                    print(f"Error reading image: {img_path}")
                    continue

                cropped_img = img[y1:y2, :]

                output_path = os.path.join(output_dir, filename)
                cv2.imwrite(output_path, cropped_img)

print("Done!")

# Renaming the Frames

In [2]:
base_dir = r'Dataset'

def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() 
            for text in re.split(r'(\d+)', s)]

In [3]:
for folder in tqdm(os.listdir(base_dir), desc="Processing folders", unit="folder"):
    folder_path = os.path.join(base_dir, folder)
    
    if not os.path.isdir(folder_path):
        continue

    img_files = [f for f in os.listdir(folder_path) 
                if f.lower().endswith(('.jpg', '.jpeg', '.png'))
                and not re.match(r'^frame_\d{4}\..+$', f)]
    
    if not img_files:
        continue

    img_files.sort(key=natural_sort_key)
    temp_dir = os.path.join(folder_path, "temp_renumber")
    os.makedirs(temp_dir, exist_ok=True)

    for filename in tqdm(img_files, desc=f"Moving '{folder}' files", unit="file"):
        src = os.path.join(folder_path, filename)
        dst = os.path.join(temp_dir, filename)
        os.rename(src, dst)

    temp_files = sorted(os.listdir(temp_dir), key=natural_sort_key)
    for idx, filename in enumerate(tqdm(temp_files, desc=f"Renaming '{folder}'", unit="file"), 1):
        ext = os.path.splitext(filename)[1].lower()
        new_name = f"frame_{idx:04d}{ext}"
        os.rename(os.path.join(temp_dir, filename),
                 os.path.join(folder_path, new_name))

    os.rmdir(temp_dir)

print("Done!")

Processing folders: 100%|██████████| 12/12 [00:10<00:00,  1.11folder/s]

Done!
